In [919]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers
!pip install -q tensorflow-hub
import tensorflow_hub as hub

In [920]:
train_limpio = pd.read_csv("train_limpio.csv",encoding = "ISO-8859-1")
train_limpio.head(2)

,text,target,@,cantidad_links,cantidad_signos_pregunta,repeticiones,longitud_tweet,letras_seguidas,sentimiento,objetividad,...,being a,boat capsizes,but if,cant wait,car engulfed,chinaÂÂs stock,city of,collapse at,insultos_por_tweet,expresiones_por_tweet
0,our deeds are the reason of this earthquake ma...,1,0,0,0,1,70,2,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,forest fire near la ronge sask canada,1,0,0,0,1,40,1,0.1,0.4,...,0,0,0,0,0,0,0,0,0,0


In [921]:
target = train_limpio["target"]
target.head(2)

0    1
1    1
Name: target, dtype: int64

In [922]:
VALIDATION_SIZE = .8
TEST_SIZE = .9
df = train_limpio[["text", "target"]]
raw_train_ds, raw_val_ds, raw_test_ds = np.split(df.sample(frac=1), [int(VALIDATION_SIZE*len(df)), int(TEST_SIZE*len(df))])

In [923]:
raw_train_ds.head(2)

,text,target
6077,arrestpastornganga it so worrying see how som...,0
6028,after a suicide bombing in suruÂÂ¤ that kille...,1


In [924]:
raw_train_ds = tf.data.Dataset.from_tensor_slices((raw_train_ds["text"].to_numpy(dtype="str"), raw_train_ds["target"].values))
raw_val_ds = tf.data.Dataset.from_tensor_slices((raw_val_ds["text"].to_numpy(dtype="str"), raw_val_ds["target"].values))
raw_test_ds = tf.data.Dataset.from_tensor_slices((raw_test_ds["text"].to_numpy(dtype="str"), raw_test_ds["target"].values))


train_examples_batch, train_labels_batch = next(iter(raw_train_ds.batch(10)))

In [925]:
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'arrestpastornganga it so worrying  see how some police officers in kenya have sunk to low to the point of collaborating with pick pockets',
       b'after a suicide bombing in suru\xc3\x82\xc2\x8c\xc3\x82\xc2\xa4 that killed  people turkey launches airstrikes against isil and kurdistan workers party camps in iraq',
       b'tsunamiesh esh please okay',
       b'modelbubbles gm i pray any attack of the enemy  derail ur destiny is blocked by the lord amp that he floods ur life wheavenly blessings',
       b'and the fact that i have a curfew',
       b'blog rain much needed as drought conditions worsen right now charlotte and much of the surrounding area have\xc3\x82\xc2\x89\xc3\x82\xc2\x9b httptcoolzavtjfkh',
       b'arson suspect linked to  fires caught in northern california httptcommgsyahdzb',
       b'freemurphy your hot take on canadas hitchhiking garbagebot destroyed in philly was sorely needed',
       b' anniversary of hiros

In [926]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 1, 0, 0, 0, 1, 1, 1, 1, 0])>

In [927]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [928]:
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.422505  , -1.0885118 ,  0.45500493,  0.62994087, -0.5561043 ,
        -2.3073676 , -0.66064364,  0.77007073, -0.32066774,  0.23901324,
        -0.19350646,  0.42796662, -0.43504363, -0.08390167, -1.7031976 ,
         1.3709724 ,  2.0736885 , -1.639749  , -1.4875039 , -0.28356174],
       [-0.08792715,  1.282691  ,  0.18003497, -0.71806717,  1.95289   ,
        -1.185932  ,  0.10456593,  0.5421631 ,  0.2026949 ,  0.64120424,
         0.4748855 , -0.32000747, -2.2775247 , -0.21664838, -1.7203114 ,
         2.1113758 , -0.8083178 , -1.7622619 , -1.9224905 ,  0.89049375],
       [ 0.08051787, -0.84861565,  0.786759  ,  1.0061411 , -1.3216624 ,
        -1.0445361 , -0.23381115,  0.5371373 ,  0.3768555 ,  0.46945632,
        -1.2872784 , -0.50762063,  0.7714113 ,  0.44169393, -0.293678  ,
        -0.5618623 ,  0.9480736 ,  0.2703368 , -0.4546466 , -0.93508834]],
      dtype=float32)>

### Esto es el modelo y, aunque parezca lo más dificil, es lo que más se entiende hasta ahora

In [929]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(raw_train_ds.shuffle(5000).batch(512),
                    epochs=22,
                    validation_data=raw_val_ds.batch(512), verbose=2)

Epoch 1/22
12/12 - 0s - loss: 0.6887 - accuracy: 0.5884 - val_loss: 0.6760 - val_accuracy: 0.5462
Epoch 2/22
12/12 - 0s - loss: 0.6328 - accuracy: 0.5981 - val_loss: 0.6319 - val_accuracy: 0.5546
Epoch 3/22
12/12 - 0s - loss: 0.6020 - accuracy: 0.6226 - val_loss: 0.6046 - val_accuracy: 0.5966
Epoch 4/22
12/12 - 0s - loss: 0.5784 - accuracy: 0.6527 - val_loss: 0.5849 - val_accuracy: 0.6373
Epoch 5/22
12/12 - 0s - loss: 0.5596 - accuracy: 0.6799 - val_loss: 0.5687 - val_accuracy: 0.6723
Epoch 6/22
12/12 - 0s - loss: 0.5439 - accuracy: 0.7084 - val_loss: 0.5540 - val_accuracy: 0.7003
Epoch 7/22
12/12 - 0s - loss: 0.5297 - accuracy: 0.7256 - val_loss: 0.5424 - val_accuracy: 0.7171
Epoch 8/22
12/12 - 0s - loss: 0.5171 - accuracy: 0.7408 - val_loss: 0.5305 - val_accuracy: 0.7339
Epoch 9/22
12/12 - 0s - loss: 0.5053 - accuracy: 0.7538 - val_loss: 0.5218 - val_accuracy: 0.7395
Epoch 10/22
12/12 - 0s - loss: 0.4947 - accuracy: 0.7627 - val_loss: 0.5139 - val_accuracy: 0.7437
Epoch 11/22
12/12 -

In [930]:
model.evaluate(raw_test_ds.batch(512))

2/2 [==============================] - 0s 2ms/step - loss: 0.4861 - accuracy: 0.7689


[0.4861498475074768, 0.7689075469970703]

### Esto ultimo significa que no sabe que predecir :)